## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,50.81,90,55,2.37,moderate rain
1,1,Popondetta,PG,-8.7537,148.2534,74.44,92,97,2.93,overcast clouds
2,2,New Norfolk,AU,-42.7826,147.0587,42.71,77,23,2.39,few clouds
3,3,Ostersund,SE,63.1792,14.6357,39.65,60,75,10.36,broken clouds
4,4,Hasaki,JP,35.7333,140.8333,61.20,49,100,12.77,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Popondetta,PG,-8.7537,148.2534,74.44,92,97,2.93,overcast clouds
4,4,Hasaki,JP,35.7333,140.8333,61.20,49,100,12.77,overcast clouds
8,8,Lock Haven,US,41.1370,-77.4469,64.90,37,0,9.53,clear sky
10,10,Antalaha,MG,-14.9003,50.2788,75.34,89,75,5.48,light rain
12,12,Los Llanos De Aridane,ES,28.6585,-17.9182,67.75,68,75,14.97,broken clouds
14,14,Vaini,TO,-21.2000,-175.2000,73.56,88,20,6.91,few clouds
16,16,Morondava,MG,-20.2833,44.2833,77.92,76,0,12.01,clear sky
19,19,Hilo,US,19.7297,-155.0900,82.27,93,100,5.75,light rain
21,21,Baghdad,IQ,33.3406,44.4009,76.91,27,0,5.75,clear sky
22,22,Rikitea,PF,-23.1203,-134.9692,76.48,79,44,22.68,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                329
City                   329
Country                326
Lat                    329
Lng                    329
Max Temp               329
Humidity               329
Cloudiness             329
Wind Speed             329
Current Description    329
dtype: int64

In [6]:
preferred_cities_df.isnull().count()

City_ID                329
City                   329
Country                329
Lat                    329
Lng                    329
Max Temp               329
Humidity               329
Cloudiness             329
Wind Speed             329
Current Description    329
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Popondetta,PG,74.44,overcast clouds,-8.7537,148.2534,
4,Hasaki,JP,61.20,overcast clouds,35.7333,140.8333,
8,Lock Haven,US,64.90,clear sky,41.1370,-77.4469,
10,Antalaha,MG,75.34,light rain,-14.9003,50.2788,
12,Los Llanos De Aridane,ES,67.75,broken clouds,28.6585,-17.9182,
14,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
16,Morondava,MG,77.92,clear sky,-20.2833,44.2833,
19,Hilo,US,82.27,light rain,19.7297,-155.0900,
21,Baghdad,IQ,76.91,clear sky,33.3406,44.4009,
22,Rikitea,PF,76.48,light rain,-23.1203,-134.9692,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# Verify hotel data.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Popondetta,PG,74.44,overcast clouds,-8.7537,148.2534,Timos Transit House
4,Hasaki,JP,61.20,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
8,Lock Haven,US,64.90,clear sky,41.1370,-77.4469,Fairfield Inn & Suites by Marriott Lock Haven
10,Antalaha,MG,75.34,light rain,-14.9003,50.2788,Hotel Vitasoa
12,Los Llanos De Aridane,ES,67.75,broken clouds,28.6585,-17.9182,Valle Aridane


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))